In [1]:
%pwd

'/workspaces/Employee-Performance-Prediction/notebook'

In [2]:
import os
os.chdir('../')
%pwd

'/workspaces/Employee-Performance-Prediction'

In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_path: Path
    target_column: str

In [8]:
from src.constants import *
from src.utils.common import read_yaml, create_directories

In [9]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config=self.config.model_trainer
        params=self.params.model_trainer
        schema=self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path=config.train_data_path,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            target_column=schema.name
        )
        return model_trainer_config

In [10]:
import os
import pandas as pd
import joblib
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from src import logger

In [15]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        try:
            # Ensure the directory for saving the model exists
            model_dir = self.config.model_path
            if not os.path.exists(model_dir):
                os.makedirs(model_dir)  # Create directory if it doesn't exist

            # Load the training and testing data
            train_data = pd.read_csv(self.config.train_data_path)
            test_data = pd.read_csv(self.config.test_data_path)

            logger.info(f"Training data loaded from {self.config.train_data_path}, shape: {train_data.shape}")
            logger.info(f"Testing data loaded from {self.config.test_data_path}, shape: {test_data.shape}")

            # Split the training data into features (X) and target (y)
            X_train = train_data.drop(columns=[self.config.target_column])
            y_train = train_data[self.config.target_column]
            
            # Split the test data into features (X) and target (y)
            X_test = test_data.drop(columns=[self.config.target_column])
            y_test = test_data[self.config.target_column]

            # Initialize the Linear Regression model
            model = LinearRegression()

            # Train the model
            model.fit(X_train, y_train)
            logger.info("Model training completed successfully.")

            # Evaluate the model
            y_pred = model.predict(X_test)
            mse = mean_squared_error(y_test, y_pred)
            logger.info(f"Model Mean Squared Error: {mse}")

            # Save the trained model to the specified path
            model_save_path = os.path.join(model_dir, 'linear_regression_model.joblib')
            joblib.dump(model, model_save_path)
            logger.info(f"Model saved at {model_save_path}")
            
        except Exception as e:
            logger.error(f"Error during model training: {e}")
            raise e


ModuleNotFoundError: No module named 'src.logger'

In [16]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(config=model_trainer_config)
    model_trainer.train()
except Exception as e:
    raise e

[2025-01-17 13:38:35,027: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-01-17 13:38:35,028: INFO: common: yaml file: params.yaml loaded successfully]
[2025-01-17 13:38:35,031: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-01-17 13:38:35,032: INFO: common: created directory at: artifacts]
[2025-01-17 13:38:35,033: INFO: common: created directory at: artifacts/model_trainer]
[2025-01-17 13:38:35,035: INFO: 239719525: Training data loaded from artifacts/data_transformation/train.csv, shape: (898, 3)]
[2025-01-17 13:38:35,036: INFO: 239719525: Testing data loaded from artifacts/data_transformation/test.csv, shape: (300, 3)]
[2025-01-17 13:38:35,039: INFO: 239719525: Model training completed successfully.]
[2025-01-17 13:38:35,041: INFO: 239719525: Model Mean Squared Error: 0.2532613175686623]
[2025-01-17 13:38:35,042: INFO: 239719525: Model saved at artifacts/model_trainer/linear_regression_model.joblib]
